# Building GPT from scratch.

Building GPT2 is a complex task but I guess I can learn about transformers. For now on, I don't really understand about `attention` and no basics with `RNN`, `LSTM`. It might be a challenging task to understand the transformers of course. But, I have built ViT (Vision Transformer). Why don't I give it a shot?

* References

  * Illustrated-gpt2 by jay alammar : https://jalammar.github.io/illustrated-gpt2/#part-1-got-and-language-modeling
  
  * Here's how you can build and train GPT-2 : https://dev.to/amit_kharel_aae65abe2b111/heres-how-you-can-build-and-train-gpt-2-from-scratch-using-pytorch-345n

I've used jay alammar's blog post to understand the architecture and how gpt2 works and bottom blog post is for dataset and preprocessing.

I want to make GPT from scratch using 140,000 korean article datasets. I've already tokenized using kkma and KR-Bert. GPT needs MaskedMultiHeadSelfAttention, MultiHeadSelfAttention and MultiLayerPerceptron. And, I know how to make 2 of them.


<img src="images/transformers.png" width="1000" height="600">


In [1]:
import torch
import torch.nn
from tqdm.auto import tqdm

from torch import nn
from transformers import AutoTokenizer
print(f"torch version : {torch.__version__}")

torch version : 2.5.1


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from Artificial_Intelligence.pytorch_modules.pytorch_modules import data_setup, engine
    from Artificial_Intelligence.helper_functions import download_data, set_seeds, plot_loss_curves
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular or helper_functions scripts... downloading them from GitHub.")
    !git clone https://github.com/DutchVandaline/Artificial_Intelligence.git
    !mv Artificial_Intelligence/pytorch_modules .
    !mv Artificial_Intelligence/helper_functions.py . # get the helper_functions.py script
    !rm -rf pytorch-deep-learning
    from Artificial_Intelligence.pytorch_modules.pytorch_modules import data_setup, engine
    from Artificial_Intelligence.helper_functions import download_data, set_seeds, plot_loss_curves

## Making Transformer Layers
- MSA layer need Mask. Attention Mask is the biggest range and inside, there are `padding mask` and `casual mask`. Mask is for focusing or ignoring the token.
  - **Padding Mask** : Removing the padding of tokenizer.
  - **Casual Mask** : Shown as Upper Triangular Mask. For Auto-Regression Model making it not see the future mask. Example is like following.
  ```
  [[0, -inf, -inf, -inf, -inf],
   [0,    0, -inf, -inf, -inf],
   [0,    0,    0, -inf, -inf],
   [0,    0,    0,    0, -inf],
   [0,    0,    0,    0,    0]]
  ```
I've used both `padding mask` and `casual mask` as `attention mask`.

In [4]:
# Not used on GPT
class MultiHeadSelfAttentionBlock(nn.Module):

  def __init__(self,
               embedding_dim:int=768,
               num_heads:int=12,
               attn_dropout:int=0):

    super().__init__()
    self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)

    self.multihead_attn= nn.MultiheadAttention(embed_dim=embedding_dim,
                      num_heads=num_heads,
                      dropout=attn_dropout,
                      batch_first=True)

  def forward(self, x):
    x = self.layer_norm(x)
    attn_output, _ = self.multihead_attn(query=x,
                                    key=x,
                                    value=x,
                                    need_weights=False)

    return attn_output


In [5]:
class MLPBlock(nn.Module):
  def __init__(self,
           embedding_dim:int=768,
           mlp_size:int=3072,
           dropout:float=0.1):
    super().__init__()
    self.layer_norm = nn.LayerNorm(normalized_shape=embedding_dim)
    self.mlp = nn.Sequential(
        nn.Linear(in_features=embedding_dim,
                  out_features=mlp_size),
        nn.GELU(),
        nn.Dropout(p=dropout),
        nn.Linear(in_features=mlp_size,
                  out_features=embedding_dim),
        nn.Dropout(p=dropout))

  def forward(self, x):
    x = self.layer_norm(x)
    x = self.mlp(x)
    return x

In [6]:
import torch
import torch.nn as nn

class MaskedMultiHeadSelfAttentionBlock(nn.Module):
    def __init__(self,
                 embedding_dim: int = 768,
                 num_heads: int = 12,
                 attn_dropout: float = 0.1):
        super().__init__()
        self.layer_norm = nn.LayerNorm(embedding_dim)

        self.multihead_attn = nn.MultiheadAttention(embed_dim=embedding_dim,
                                                   num_heads=num_heads,
                                                   dropout=attn_dropout,
                                                   batch_first=True)

    def forward(self, x, key_padding_mask=None):
        # Normalize input
        x = self.layer_norm(x)

        # Masked Self-Attention
        batch_size, seq_len, _ = x.size()

        # Create causal mask (lower triangular matrix for self-attention)
        casual_mask = torch.tril(torch.ones(seq_len, seq_len, device=x.device))

        # Apply MultiheadAttention
        attn_output, _ = self.multihead_attn(
            query=x,
            key=x,
            value=x,
            attn_mask=casual_mask,  # Causal mask for self-attention
            key_padding_mask=key_padding_mask,  # Padding mask
            need_weights=False
        )
        
        return attn_output


## Making TransformerDecoderBlock
GPT Transformer block doesn't use MSA block. It uses Masked MultiheadSelfAttentionLayer.

In [7]:
class TransformerDecoderBlock(nn.Module):
    def __init__(self,
                 embedding_dim: int = 768,
                 num_heads: int = 12,
                 mlp_size: int = 3072,
                 mlp_dropout: float = 0.1,
                 attn_dropout: float = 0.1):
        super().__init__()

        # Create Masked Self-Attention block (for autoregressive behavior)
        self.masked_msa_block = MaskedMultiHeadSelfAttentionBlock(
            embedding_dim=embedding_dim,
            num_heads=num_heads,
        )

        # Create Feed-Forward block (MLP)
        self.mlp_block = MLPBlock(
            embedding_dim=embedding_dim,
            mlp_size=mlp_size,
            dropout=mlp_dropout
        )

        # Layer normalization for each block
        self.layer_norm1 = nn.LayerNorm(embedding_dim)
        self.layer_norm2 = nn.LayerNorm(embedding_dim)

    def forward(self, x, key_padding_mask=None):
        #print(f"Before self-attention: {x.isnan().any()}")

        attn_output = self.masked_msa_block(x, key_padding_mask)
        x_residual1 = attn_output + x
        #print(f"After self-attention: {x_residual1.isnan().any()}")
        
        # Apply Feed-Forward block (MLP) with residual connection
        mlp_output = self.mlp_block(x_residual1)
        x_residual2 = mlp_output + x_residual1
        #print(f"After feed-forward: {x_residual2.isnan().any()}")
        
        return x_residual2


In [8]:
class GPTDecoder(nn.Module):
    def __init__(self,
                 num_layers: int = 12,
                 embedding_dim: int = 768,
                 num_heads: int = 12,
                 mlp_size: int = 3072,
                 mlp_dropout: float = 0.1,
                 attn_dropout: float = 0.1):
        super().__init__()

        # Stack multiple transformer decoder layers
        self.decoder_layers = nn.ModuleList([
            TransformerDecoderBlock(
                embedding_dim=embedding_dim,
                num_heads=num_heads,
                mlp_size=mlp_size,
                mlp_dropout=mlp_dropout,
                attn_dropout=attn_dropout
            ) for _ in range(num_layers)
        ])


    def forward(self, x, key_padding_mask=None):
        for layer in self.decoder_layers:
            x = layer(x, key_padding_mask)
        return x


In [9]:
class GPT(nn.Module):
    def __init__(self,
                 vocab_size: int,  # Vocabulary size
                 max_seq_len: int = 256,  # Maximum sequence length
                 embedding_dim: int = 768,
                 num_layers: int = 12,
                 num_heads: int = 12,
                 mlp_size: int = 3072,
                 mlp_dropout: float = 0.1,
                 attn_dropout: float = 0.1):
        super().__init__()

        # Token Embedding
        self.token_embedding = nn.Embedding(vocab_size, embedding_dim)

        # Positional Embedding
        self.positional_embedding = nn.Parameter(torch.zeros(1, max_seq_len, embedding_dim))

        # Decoder stack
        self.decoder = GPTDecoder(
            num_layers=num_layers,
            embedding_dim=embedding_dim,
            num_heads=num_heads,
            mlp_size=mlp_size,
            mlp_dropout=mlp_dropout,
            attn_dropout=attn_dropout
        )

        # Output projection to vocab size
        self.output_layer = nn.Linear(embedding_dim, vocab_size)

    def forward(self, input_ids, key_padding_mask=None):
        # Step 1: Embed tokens and add positional embeddings
        x = self.token_embedding(input_ids)  # Shape: [batch_size, seq_len, embedding_dim]
        #print(f"Token Embedding (x) shape: {x.shape}")
        #print(f"x contains NaN after token embedding: {x.isnan().any()}")

        seq_len = input_ids.size(1)
        x = x + self.positional_embedding[:, :seq_len, :]  # Add positional embedding
        #print(f"x after adding positional embedding: {x.shape}")
        #print(f"x contains NaN after positional embedding: {x.isnan().any()}")

        # Step 2: Check for NaN in input (if key_padding_mask is provided)
        if key_padding_mask is not None:
            key_padding_mask = key_padding_mask.to(torch.bool)  # Ensure the mask is of bool type
            #print(f"key_padding_mask contains NaN: {key_padding_mask.isnan().any()}")

        # Step 3: Pass through decoder stack
        x = self.decoder(x, key_padding_mask)
        #print(f"x after decoder: {x.shape}")
        #print(f"x contains NaN after decoder: {x.isnan().any()}")

        # Step 4: Output projection to vocab size
        logits = self.output_layer(x)
        #print(f"logits shape: {logits.shape}")
        #print(f"logits contains NaN: {logits.isnan().any()}")
        
        return logits


## Making Dataloader
Data is based on AI-Hub firstly, made of 20,000 Korean Article. Tokenizer was done by kkma and KR-Bert.

In [10]:
import os
import torch
from torch.utils.data import Dataset, DataLoader

class PreprocessedKoreanDataset(Dataset):
    def __init__(self, data_dir):
        self.data_dir = data_dir
        self.file_list = [f for f in os.listdir(data_dir) if f.endswith('.pt')]

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        file_path = os.path.join(self.data_dir, self.file_list[idx])
        data = torch.load(file_path, weights_only=True)

        input_ids = data['input_ids']
        attention_mask = data['attention_mask']

        # Create key_padding_mask based on attention_mask
        # Key padding mask is 1 for padded tokens, 0 for non-padded tokens
        key_padding_mask = (attention_mask == 0).bool()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'key_padding_mask': key_padding_mask
        }

#data_dir = "krbert_korean_pretrain"
data_dir = "C:/junha/Git_Clone/LLM_Classifier/data/training/training/korean"
batch_size = 16

# Create Dataset and DataLoader
dataset = PreprocessedKoreanDataset(data_dir)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example of iterating over the DataLoader
for batch in dataloader:
    input_ids = batch['input_ids']
    attention_mask = batch['attention_mask']
    key_padding_mask = batch['key_padding_mask']

print(input_ids.shape, attention_mask.shape, key_padding_mask.shape)


torch.Size([8, 256]) torch.Size([8, 256]) torch.Size([8, 256])


## Train Model

In [11]:
krbert_tokenizer = AutoTokenizer.from_pretrained("snunlp/KR-Medium", do_lower_case=False)

In [12]:
print(f"Input IDs Min: {input_ids.min()}, Max: {input_ids.max()}")
print(f"Padding Token ID: {krbert_tokenizer.pad_token_id}")

print(f"Input IDs Example: {input_ids[1]}")
print(f"Attention Mask Example: {attention_mask[1]}")

Input IDs Min: 0, Max: 19887
Padding Token ID: 0
Input IDs Example: tensor([    2,  8603,  9235,    26,  2304,  3974,  8933,  5014,  2995,  3382,
         5207, 16160,  2780, 15015,  4492,  2007,  3758,  2367,  2104,  3678,
         2897, 14945,  3504,  2367,  9226,   417,  8823,  3749,  1932,  5364,
         5028,  3737,  4059,  2007,  5034,  3585,  9570,  2489,   417, 11810,
         4157,  5023,  9558, 11075,  3721,  9041,  9668,  8802,  4492,  3565,
         3030,  3529,  2396,    18, 12292, 15015,  3807,  2104,  3504,  2367,
        16257,  5361,  9570,  2489,   417,  3940,  5142, 12388,  2099,  3585,
        12713,  3721, 10368,  5076,  2007,  4157,  5023,  5056,  3585,  3325,
         3721,  3066,  5043,  5809,  3766,  2871,  5071,  5248,  4578,  5093,
         3585,  4560,  3898,  3520,  5199,  5719,  2323,  5067,    20,  3742,
         9718,  9092,  3742, 13801,  2489,  3579,  2396,    18,  2104,  3504,
         2367,  9554, 12292,  8583,  9222,  4560,  3898,  3520,  5199,  57

In [13]:
batch_size = 16
vocab_size = krbert_tokenizer.vocab_size

gpt = GPT(vocab_size=vocab_size).to(device)
loss_fn = nn.CrossEntropyLoss(ignore_index=krbert_tokenizer.pad_token_id)
optimizer = torch.optim.Adam(gpt.parameters(), lr=1e-5,weight_decay=0.01)

In [14]:
def train_step(model, dataloader, loss_fn, optimizer, device):
    model.train()
    train_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        logits = model(input_ids, key_padding_mask=attention_mask)

        logits = logits.view(-1, logits.size(-1))
        labels = input_ids.view(-1).to(device)  # 예측할 레이블은 input_ids 자체

        loss = loss_fn(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        _, predicted = torch.max(logits, dim=-1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

    # 평균 손실과 정확도 계산
    avg_loss = train_loss / len(dataloader)
    accuracy = correct_preds / total_preds
    return avg_loss, accuracy

In [17]:
epochs = 20

for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(
        model=gpt,
        dataloader=dataloader,
        loss_fn=loss_fn,
        optimizer=optimizer,
        device=device
    )
    print(f"Epoch {epoch + 1} - Loss: {train_loss:.4f}, Accuracy: {train_acc:.4f}")


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch 1 - Loss: 0.4650, Accuracy: 0.8632
Epoch 2 - Loss: 0.5439, Accuracy: 0.8554
Epoch 3 - Loss: 0.6652, Accuracy: 0.8434
Epoch 4 - Loss: 0.7967, Accuracy: 0.8282
Epoch 5 - Loss: 0.8199, Accuracy: 0.8150
Epoch 6 - Loss: 0.7668, Accuracy: 0.8127
Epoch 7 - Loss: 0.6971, Accuracy: 0.8165
Epoch 8 - Loss: 0.6336, Accuracy: 0.8234
Epoch 9 - Loss: 0.5791, Accuracy: 0.8308
Epoch 10 - Loss: 0.5327, Accuracy: 0.8380
Epoch 11 - Loss: 0.4845, Accuracy: 0.8452
Epoch 12 - Loss: 0.4382, Accuracy: 0.8519
Epoch 13 - Loss: 0.4025, Accuracy: 0.8570
Epoch 14 - Loss: 0.3697, Accuracy: 0.8612
Epoch 15 - Loss: 0.3392, Accuracy: 0.8654
Epoch 16 - Loss: 0.3171, Accuracy: 0.8687
Epoch 17 - Loss: 0.3014, Accuracy: 0.8711
Epoch 18 - Loss: 0.2872, Accuracy: 0.8734
Epoch 19 - Loss: 0.2730, Accuracy: 0.8757
Epoch 20 - Loss: 0.2641, Accuracy: 0.8770


## Generate?

In [18]:
from Artificial_Intelligence.pytorch_modules.pytorch_modules import utils

utils.save_model(model=gpt,
                 target_dir="C:/junha/GPT_from_Scratch/models",
                 model_name="GPTScratch_300K.pth")

[INFO] Saving model to: C:\junha\GPT_from_Scratch\models\GPTScratch_300K.pth


In [20]:
import torch
import torch.nn.functional as F

class GPTWithGenerate(GPT):
    def generate(self, input_ids, max_length=50, num_return_sequences=1, temperature=1.0, top_k=50):
        self.eval()  # 평가 모드로 설정
        
        batch_size = input_ids.size(0)
        generated_sequences = []

        # 여러 시퀀스를 생성할 경우
        for _ in range(num_return_sequences):
            generated_ids = input_ids  # 초기 입력 시퀀스
            for _ in range(max_length):
                # Step 1: 모델의 출력을 얻기 위해 forward 패스 실행
                outputs = self(input_ids=generated_ids)
                logits = outputs  # 모델의 logits, shape: [batch_size, seq_len, vocab_size]
                logits = logits[:, -1, :] / temperature  # 마지막 위치의 logits만 사용하고, temperature로 스케일링
                
                # Step 2: 확률로 변환 (softmax)
                next_token_probs = F.softmax(logits, dim=-1)
                
                # Step 3: top-k 샘플링 (선택적)
                if top_k > 0:
                    top_k_probs, top_k_indices = torch.topk(next_token_probs, top_k, dim=-1)
                    next_token = torch.multinomial(top_k_probs, 1)  # top-k 확률 분포에서 샘플링
                    next_token = top_k_indices.gather(-1, next_token)
                else:
                    next_token = torch.argmax(next_token_probs, dim=-1).unsqueeze(-1)  # argmax로 다음 토큰 예측

                # Step 4: 예측된 토큰을 입력 시퀀스에 추가
                generated_ids = torch.cat((generated_ids, next_token), dim=-1)

            generated_sequences.append(generated_ids)

        return generated_sequences

# 기존 GPT 모델을 GPTWithGenerate 클래스로 확장하여 사용
gptScratch = GPTWithGenerate(vocab_size=vocab_size).to(device)
gptScratch.load_state_dict(torch.load("C:/junha/GPT_from_Scratch/models/GPTScratch_300K.pth", weights_only=True))

<All keys matched successfully>

In [25]:
prompt = "오늘 날씨가 "
input_ids = krbert_tokenizer(prompt, return_tensors="pt").input_ids.to(device)

generated_sequences = gptScratch.generate(input_ids, max_length=50, num_return_sequences=1, temperature=1.0, top_k=50)
print(generated_sequences)
generated_text = krbert_tokenizer.decode(generated_sequences[0][0], skip_special_tokens=True)
print(generated_text)


[tensor([[    2,  9062,  2244, 10223,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
             3,     3,     3,     3,     3,     3,     3,  9472,  9472,  9472,
          9472,  9472,  5031,  5031,  5031]], device='cuda:0')]
오늘 날씨가 가격 가격 가격 가격 가격미미미


In [ ]:
def load_model_checkpoint(checkpoint_path, model, optimizer=None):
    checkpoint = torch.load(checkpoint_path)
    

    model.load_state_dict(checkpoint['model_state_dict'])
    
    if optimizer:
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
    print(f"Model loaded from {checkpoint_path}")
    print(f"Last trained epoch: {epoch}, Loss: {loss:.4f}")
    
    return epoch, loss

vocab_size = krbert_tokenizer.vocab_size
gpt = GPT(vocab_size=vocab_size).to(device)
optimizer = torch.optim.Adam(gpt.parameters(), lr=1e-5)


checkpoint_path = "/home/junha/GPT_from_Scratch/checkpoints/gpt_final_epoch.pt"

start_epoch, last_loss = load_model_checkpoint(checkpoint_path, gpt, optimizer)
